In [1]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

# 参考ノートブック
https://www.kaggle.com/foreveryoung/least-squares-solution-from-gnss-derived-data

# インポート

In [57]:
import pandas as pd
import numpy as np
import math

# ユーティリティ関数

In [3]:
def ecef2lla(x, y, z):
  # x, y and z are scalars or vectors in meters
  x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
  y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
  z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

  a=6378137
  a_sq=a**2
  e = 8.181919084261345e-2
  e_sq = 6.69437999014e-3

  f = 1/298.257223563
  b = a*(1-f)

  # calculations:
  r = np.sqrt(x**2 + y**2)
  ep_sq  = (a**2-b**2)/b**2
  ee = (a**2-b**2)
  f = (54*b**2)*(z**2)
  g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
  c = (e_sq**2)*f*r**2/(g**3)
  s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
  p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
  q = np.sqrt(1 + 2*p*e_sq**2)
  r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
  u = np.sqrt((r - e_sq*r_0)**2 + z**2)
  v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
  z_0 = (b**2)*z/(a*v)
  h = u*(1 - b**2/(a*v))
  phi = np.arctan((z + ep_sq*z_0)/r)
  lambd = np.arctan2(y, x)

  return phi*180/np.pi, lambd*180/np.pi, h

# load provided data

In [34]:
datapath = "/work/data/input/google-smartphone-decimeter-challenge"

df_baseline_train = pd.read_csv(f"{datapath}/baseline_locations_train.csv")
df_sample_trail_gt = pd.read_csv(f"{datapath}/train/2020-05-14-US-MTV-1/Pixel4/ground_truth.csv")    
df_sample_trail = pd.read_csv(f"{datapath}/train/2020-05-14-US-MTV-1/Pixel4/Pixel4_derived.csv")

df_sample_trail["correctedPrM"] = df_sample_trail["rawPrM"] + df_sample_trail["satClkBiasM"] - df_sample_trail["isrbM"] - df_sample_trail["ionoDelayM"] - df_sample_trail["tropoDelayM"] 


sat_pos = df_sample_epoch[["xSatPosM","ySatPosM","zSatPosM"]].to_numpy()
pseudoranges = np.squeeze(df_sample_epoch[["correctedPrM"]].to_numpy())

In [5]:
df_sample_trail.head(3)

,collectionName,phoneName,millisSinceGpsEpoch,constellationType,svid,signalType,receivedSvTimeInGpsNanos,xSatPosM,ySatPosM,zSatPosM,...,ySatVelMps,zSatVelMps,satClkBiasM,satClkDriftMps,rawPrM,rawPrUncM,isrbM,ionoDelayM,tropoDelayM,correctedPrM
0,2020-05-14-US-MTV-1,Pixel4,1273529464442,3,24,GLO_G1,1273529463363061857,-2.539901e+07,-6.925122e+05,-2.280430e+06,...,156.040,3559.757,-468.084,0.001,2.379498e+07,11.992,1134.758,10.866,16.647,2.379335e+07
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,6,13,GAL_E1,1273529463363970742,-5.199894e+06,-1.741927e+07,2.336128e+07,...,700.815,1022.014,120171.076,0.000,2.352251e+07,1.799,-222.675,3.946,2.717,2.364289e+07
2,2020-05-14-US-MTV-1,Pixel4,1273529464442,1,5,GPS_L1,1273529463365539137,-2.179863e+06,-2.615488e+07,-3.437694e+06,...,-419.725,3129.012,-3793.067,-0.001,2.305231e+07,4.197,0.000,7.554,5.704,2.304851e+07


# 理論


## 位置推定式

初めに、第$k$個目のサテライトから見たスマホの位置$\sigma^k$は、以下式で求められる。

$$ \sigma^k = r_u^k + b_u - Pr_u^k  $$

ここで、$r_u^k$は第$k$サテライトとスマホ間の距離、$b_u$は、ユーザクロックバイアスを距離変換したもの、$Pr_u^k$は？？？である。



## サテライト・スマホ間の距離 $r_u^k$

$x^k_{sat}$と$x_u$はそれぞれ、サテライト・スマホの正確な座標位置である。

$$r_u^k = ||x^k_{sat} - x_u||$$


## ユーザクロックバイアスを距離変換したもの $b_u$


初めに、ユーザクロックバイアス$b_u$とは、GPSの送受信端末の時間誤差を距離変換したものであり、クロックバイアスを$clock\_bias_u$、光速を$c$としたとき、以下式で算出される。

$$b_u = clock\_bias_u * c$$

クロックバイアスに関して、詳細は[こちら](https://www.jstage.jst.go.jp/article/ieiej/31/5/31_340/_pdf/-char/ja)で確認してください。


## 補正範囲 


$$correctedPrM = rawPrM + satClkBiasM - isrbM - ionoDelayM - tropoDelayM$$

右辺の各変数は、"*_derived"で記載のファイルにある変数である。







<img src='https://www.researchgate.net/publication/324989880/figure/fig1/AS:623672356253699@1525706485241/Global-Navigation-Satellite-System-GNSS-spoofing-attack-illustration.png' width=600/>

# 実装

## 自作関数の場合

### 関数定義

In [117]:
def least_squares(sat_pos, pseudoranges, weights=1, x_hat=np.array([0, 0, 0, 0])):
    """
    Args:
    sat_pos: The satellite position (meters) in an ECEF coordinate frame
    pseudoranges: The corrected pseudorange (i.e. a closer approximation to the geometric range from the phone to the satellite)
    x_hat: the phone's initial/previous estimated position (x, y, z, b) and 
           b represent the user clock bias in units of distance = clock bias (t) * light speed (c)
           
    Returns:
     x_hat: the phone's estimated position
    norm_dp:
    """
    sat_pos = rotate_sat(sat_pos,0.07)
    
    dx = np.Inf*np.ones(3);
    G = np.ones((pseudoranges.size, 4))
    iterations = 0
    
    if isinstance(weights, np.ndarray):
        weights = np.diag(weights)
    else:
        weights = weights*np.eye(pseudoranges.size)

    while np.linalg.norm(dx) > 1e-3:
        norms = np.linalg.norm(sat_pos - x_hat[:3], axis=1)
        dp = pseudoranges - norms - x_hat[3]
        G[:, 0:3] = -(sat_pos - x_hat[:3])/norms[:, None]
        # G_T = np.transpose(G)
        # dx = np.linalg.inv(G_T@G) @ G_T @ dp
        dx = np.linalg.pinv(weights@G) @ weights @ dp
        x_hat = x_hat + dx
        iterations += 1
    return x_hat, np.linalg.norm(dp)

### 1エポック分のデータ抽出

In [80]:
epoch_time = 1273529464442

df_sample_epoch = df_sample_trail[df_sample_trail["millisSinceGpsEpoch"] == epoch_time]
df_sample_epoch_gt = df_sample_trail_gt[df_sample_trail_gt["millisSinceGpsEpoch"] == epoch_time]

df_sample_epoch_baseline = df_baseline_train[(df_baseline_train.collectionName == "2020-05-14-US-MTV-1") & (df_baseline_train.phoneName == "Pixel4") & (df_baseline_train.millisSinceGpsEpoch == epoch_time)]

sat_pos = df_sample_epoch[["xSatPosM","ySatPosM","zSatPosM"]].to_numpy()

display(df_sample_epoch.head(3))

pseudoranges = np.squeeze(df_sample_epoch[["correctedPrM"]].to_numpy())

,collectionName,phoneName,millisSinceGpsEpoch,constellationType,svid,signalType,receivedSvTimeInGpsNanos,xSatPosM,ySatPosM,zSatPosM,...,ySatVelMps,zSatVelMps,satClkBiasM,satClkDriftMps,rawPrM,rawPrUncM,isrbM,ionoDelayM,tropoDelayM,correctedPrM
0,2020-05-14-US-MTV-1,Pixel4,1273529464442,3,24,GLO_G1,1273529463363061857,-2.539901e+07,-6.925122e+05,-2.280430e+06,...,156.040,3559.757,-468.084,0.001,2.379498e+07,11.992,1134.758,10.866,16.647,2.379335e+07
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,6,13,GAL_E1,1273529463363970742,-5.199894e+06,-1.741927e+07,2.336128e+07,...,700.815,1022.014,120171.076,0.000,2.352251e+07,1.799,-222.675,3.946,2.717,2.364289e+07
2,2020-05-14-US-MTV-1,Pixel4,1273529464442,1,5,GPS_L1,1273529463365539137,-2.179863e+06,-2.615488e+07,-3.437694e+06,...,-419.725,3129.012,-3793.067,-0.001,2.305231e+07,4.197,0.000,7.554,5.704,2.304851e+07


### 推定

In [88]:
x, dp = least_squares(sat_pos, pseudoranges)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Simple Least Square Estimation:", *ecef2lla(*x[:3]))

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Simple Least Square Estimation: [[37.42361372]] [[-122.0936959]] [[-26.18615617]]


## Scipy optimize version

### 関数定義

In [114]:
import scipy.optimize as opt

def calc_pos_fix(sat_pos, pr, weights=1, x0 = [0,0,0,0]):
    """
    calclates gps fix with WLS optimizer
    
    return:
        0 -> list with positions
        1 -> pseudorange errs                
    """
    #sat_pos = rotate_sat(sat_pos,0.07)
    
    n = len(pr)
    
    if (n < 3):
        return x0, [];
    
    Fx_pos = pr_residual(sat_pos, pr, weights = weights);
    opt_pos = opt.least_squares(Fx_pos, x0).x
    return opt_pos, Fx_pos(opt_pos, weights=1)
    
    
def pr_residual(sat_pos, pr, weights=1):
  # solve for pos
  def Fx_pos(x_hat, weights=weights):
    rows = weights * (np.linalg.norm(sat_pos - x_hat[:3], axis=1) + x_hat[3] - pr)
    return rows
  return Fx_pos    

### 推定

In [115]:
x, dp = calc_pos_fix(sat_pos, pseudoranges)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Simple Least Square Estimation:", *ecef2lla(*x[:3]))

print("Weighted Least Square haversine distance (M):", calc_haversine(*deg_gt, *ecef2lla(*x[:3])[:2]))

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Simple Least Square Estimation: [[37.42361372]] [[-122.0936959]] [[-26.18615579]]
Weighted Least Square haversine distance (M): [[38.71773646]]


In [110]:
def rotate_sat(sat, tm):
    res = sat.copy()
    ang = math.pi*(tm)/(126060)
    res[:,2] = sat[:,2] 
    res[:,0] = np.cos(ang)*sat[:,0]+np.sin(ang)*sat[:,1] 
    res[:,1] = -np.sin(ang)*sat[:,0]+np.cos(ang)*sat[:,1]    
    return res

## Weighted Least Square Solution

### 理論

6.2節、6.3節では複数のサテライトを平等に扱い、推定を行ったが、実際には推定に悪影響を及ぼすやつもある。
なので、$rawPrUc$に基づき重み付けを行い、より推定に有益なサテライトを優先的に推定に用いる。


In [120]:
pseudoranges_sigma = np.squeeze(df_sample_epoch[["rawPrUncM"]].to_numpy())

x, dp = least_squares(sat_pos, pseudoranges, 1/pseudoranges_sigma)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation:", *ecef2lla(*x[:3]))


print("Weighted Least Square haversine distance (M):", calc_haversine(*deg_gt, *ecef2lla(*x[:3])[:2]))

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Weighted Least Square Estimation: [[37.42357917]] [[-122.09375833]] [[-35.25074978]]
Weighted Least Square haversine distance (M): [[32.98200155]]


## 評価関数

### 評価関数

In [46]:
def calc_haversine(lat1, lon1, lat2, lon2):
  """Calculates the great circle distance between two points
  on the earth. Inputs are array-like and specified in decimal degrees.
  """
  RADIUS = 6_367_000
  lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
  dlat = lat2 - lat1
  dlon = lon2 - lon1
  a = np.sin(dlat/2)**2 + \
      np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
  dist = 2 * RADIUS * np.arcsin(a**0.5)
  return dist

### 1エポック分の評価

In [109]:
deg_gt = df_sample_epoch_gt[["latDeg","lngDeg"]].to_numpy().transpose()
deg_baseline = df_sample_epoch_baseline[["latDeg","lngDeg"]].to_numpy().transpose()
print("Weighted Least Square (baseline) haversine distance (M):", calc_haversine(*deg_gt, *deg_baseline))
print("Weighted Least Square haversine distance (M):", calc_haversine(*deg_gt, *ecef2lla(*x[:3])[:2]))

Weighted Least Square (baseline) haversine distance (M): [2.74590055]
Weighted Least Square haversine distance (M): [[32.9857079]]


In [50]:
def simple_pipeline(df_trails):
  """ simple pipeline to estimate the GNSS receiver location by least square
  Args:
    df_trails: the df read from derived file
  
  Returns:
    result df with estimated degrees and heights
  """

  ### 推定に必要な
  df_trails["correctedPrM"] = df_trails["rawPrM"] + df_trails["satClkBiasM"] - df_trails["isrbM"] - df_trails["ionoDelayM"] - df_trails["tropoDelayM"]
  
  results = []
  x = [0, 0, 0, 0]
  df_epochs = df_trails.groupby(["collectionName", "phoneName", "millisSinceGpsEpoch"])
  for indices, df_epoch in tqdm(df_epochs, desc="Estimate location by LS for epoch"):
    sat_pos = df_epoch[["xSatPosM","ySatPosM","zSatPosM"]].to_numpy()
    pseudoranges = np.squeeze(df_epoch[["correctedPrM"]].to_numpy())
    pseudoranges_sigma = np.squeeze(df_epoch[["rawPrUncM"]].to_numpy())
    x, _ = calc_pos_fix(sat_pos, pseudoranges, 1/pseudoranges_sigma, x)
    # x, _ = calc_pos_fix(sat_pos, pseudoranges, 1, x)
    values = np.squeeze(ecef2lla(*x[:3]))
    results.append([*indices, *values])
  return pd.DataFrame(results,columns=["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg", "heightAboveWgs84EllipsoidM"])